# Computer Science Assignment
**Course**: Computer Science <br>
**Title**: Scalable Product Duplicate Detection <br>
**Author**: Diede Boerman (617807) <br>
**Teacher**: dr. Flavius Frasincar

The goal of this assignment is to create a scalable solution for duplicate product detection. We propose a solution that reduces the required number of computations by lowering the number of comparisons. We aim to achieve this by presenting a Locality Sensitive Hashing technique (LSH). <br>
In this document we provide the code used for programming the methods needed to obtain a suitable solution. 

In [397]:
# Importing libraries
import pandas as pd 
import numpy as np
import json
from collections import Counter
import collections
from tqdm import tqdm
import string
from random import randint, random, sample
import re
import itertools
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle

In [195]:
# Loading the provided datafile 
TVset = open('TVs-all-merged.json')
 
# Return json file as dictionary
data = json.load(TVset)

In [160]:
# Create 'workable' list of json file
ModelID = list(data.keys())

N = len(ModelID) #length is 1262 and hence including duplicates
i = 0
dataframe = []

while i < N:
    if len(data[ModelID[i]]) == 1: 
        dataframe.append(data[ModelID[i]][0])
    else:
        for duplicate in data[ModelID[i]]: 
            dataframe.append(duplicate)
    i+=1

First, we extract so-called 'model words' from the titles of the products. We initialize an empty set to store all model words retrieved from the titles. This set is used later on to create a binary vector representation for each product. Also, we do some data cleaning to remove frequently observed inconsistencies. 

In [233]:
# Extract model words from product titles. 
i = 0 
MW = [] #initialize empty list for collecting model words.
Title_list = [] #initialize empty list for collecting titles. 

# Extract model words from titles of all 1624 products 
while i < N:
    if len(data[ModelID[i]]) == 1: 
        MW.extend(data[ModelID[i]][0]['title'].split()) 
    else:
        for duplicate in data[ModelID[i]]: 
            MW.extend(duplicate['title'].split())
    i+=1

# Data cleaning model words
MW = [w.lower() for w in MW]
MW = [w.replace('"', '-inch') for w in MW]
MW = [w.replace('-lcd', 'lcd') for w in MW]
MW = [w.replace('-hz', 'hz') for w in MW]
for char in string.punctuation: 
    MW = [w.strip(char) for w in MW]

# Extracting titles and clean them
i=0

while i<N:
    if len(data[ModelID[i]])==1:
        prod_title = data[ModelID[i]][0]['title'].lower()
        prod_title = prod_title.replace('"','-inch')
        prod_title = prod_title.replace('-lcd', 'lcd')
        prod_title = prod_title.replace('-hz','hz')
        Title_list.append(prod_title)
    else:
        for duplicate in data[ModelID[i]]:
            prod_title = duplicate['title'].lower()
            prod_title = prod_title.replace('"', '-inch')
            prod_title = prod_title.replace('-lcd', 'lcd')
            prod_title = prod_title.replace('-hz','hz')
            Title_list.append(prod_title)
    i+=1

Using the set of model words (*MW*), we create a binary vector representation for each product. If the title of a product contains a specific model word from *MW*, the elemnt in the binary vector corresponding to this word is set to 1 (and 0 otherwise). The results are stored in a matrix containing representations for all products. 

In [243]:
# Set columns for dataframe in kind of a cumbersome way...
words = Counter(MW) #keeps track of how many words are in the list
words = words.most_common() #keeps track of how many times equivalent words are added

binrep_words = []

for tup in words:
    if tup[1] >= 1: #add all occuring words to the list. 
        binrep_words.append(tup[0])

binrep_words.remove("")

df = pd.DataFrame(columns=binrep_words)

# Fill dataframe
new_row = []
for prod_title in Title_list:
    for element in binrep_words:
        if element in prod_title:
            new_row.append(1)
        else:
            new_row.append(0)
            
    df.loc[len(df)]= new_row
    new_row = []

Binary_Vector_Matrix = df
Binary_Vector_Matrix

,hdtv,1080p,class,led,diag,newegg.com,best,buy,3d,samsung,...,42ln5200,th-37lru5,22lv255c,45.9-inchdiagonal,un46f5000afxza,60125-inch,45.9,nt-1907,un46es7100fxza,e424
0,1,0,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1620,1,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1621,1,1,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1622,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Before obtaining an LSH algorithm, we apply min-hashing. We compress the binary vectors into signature vectors, resulting into a smaller set of elements. Each vector is assigned a signature in such a way that similar products receive the same signature. 

In [309]:
# Minhashing
HF = 60 #set hash function

datatrans = Binary_Vector_Matrix.transpose()
datatrans

signaturematrix = np.full((HF, len(datatrans.columns)), np.inf) #create new array

for i in range(HF):
    datatrans = shuffle(datatrans) #apply permutations     
        
    for prod in datatrans:
        value = list(datatrans[prod]).index(1) #find the first '1' in the row
        signaturematrix[i][prod] = value #store corresponding rownumber
        
sigmat = signaturematrix.T
sigmat

array([[ 35.,  33.,  98., ...,  20.,  23.,   2.],
       [  9., 117.,  66., ...,  67., 290., 225.],
       [ 35.,   9.,  26., ...,  20., 166.,   3.],
       ...,
       [ 35.,  33., 123., ...,  63., 124.,   3.],
       [ 35.,   9.,  26., ...,  67., 259.,   3.],
       [132., 109.,  97., ...,  20., 124.,   3.]])

### LSH algorithm
We will now apply LSH to the signature matrix. We aim to find candidate pairs out of the matrix. Creating the algorithm consists of several steps. We create separate functions in Python for this. <br>

First of all, we create the bucketmatrix. For each band $b$, we use a hash function to hash the vector into a certain bucket. LSH should hash two products to the same bucket if they are likely to be similar. 

In [304]:
# Defining function to create bucketmatrix

def LSH(M, b, r):
    n, d = M.shape
    assert(d==b*r) #continue if statement holds

    bucket_matrix = np.full((b, n), 0) #create new array   

    k=0
    for band in range(b): #loop over bands
        signaturelist = [] #create empty list

        for product in range(n): #loop over products
            partsignature = M[product, k:r+k]
            signmatch = list(partsignature % 10)
            
            if signmatch not in signaturelist: #making buckets
                signaturelist.append(signmatch)
            else:
                None

            bucket = signaturelist.index(signmatch)
            bucket_matrix[band][product] = bucket    
        
        k = k + r
        
    return bucket_matrix



In [311]:
b = 15 #set number of bands
r = 4 #set number of rows
buckmat = LSH(sigmat, b, r)
buckmat

array([[  0,   1,   2, ...,  11,   2, 269],
       [  0,   1,   2, ...,   2,  34,  34],
       [  0,   1,   2, ...,   4,  58,  10],
       ...,
       [  0,   1,   2, ...,  39,  40,  12],
       [  0,   1,   0, ...,  19,  19,   0],
       [  0,   1,   2, ..., 109, 408,   6]])

Two products that are hashed to the same bucket will be considered as a candidate pair. Hence, we now create some code for finding the candidate pairs. We make use of the bucket matrix ('buckmat) obtained from the previous function. 

In [ ]:
# Defining candidate pairs

cpairs = set() #initialize set
hb = collections.defaultdict(set)

for bandnr in range(b): #loop over all bands
    band = list(buckmat[bandnr]) 
    for bucketnr in set(band): #loop over the buckets
        for i in range(len(band)):
            if band[i]==bucketnr:  
                hb[bucketnr].add(i) #add hash bucket if band index equals bucket number
            else:
                None 
                
    for bucketnr in hb.values():
        if len(bucketnr)>1:
            for pair in itertools.combinations(bucketnr,2): #find possible combinations
                cpairs.add(pair) #label pairs as candidate pair

#cpairs

Now that we have defined the candidate pairs, we consider the problem of considering the pairs under a certain threshold value. We do this to find the right balance between the number of false positives and false negatives. As threshold we set a value of 0.51 (based on the Jaccard threshold of $\frac{1}{b}^{\frac{1}{r}}$). Two items are considered a 'real' candidate pair if the computed Jaccard similarity is bigger than the threshold value. The Jaccard similarity is computed for the signatures of the created signature matrix.

We first create a function to calculate and return a Jaccard similarity. Then, we create a set of 'real' LSH pairs, satisfying the chosen threshold value. 

In [411]:
# Function for calculating Jaccard similarity
def jacsim(one, two):
    s1 = set(one)
    s2 = set(two)
    
    return float(len(s1.intersection(s2))/len(s1.union(s2)))

testjaccard = jacsim(sigmat[1], sigmat[2])

# Store candidate pairs that meet threshold requirement
threshold = 0.51

realcpairs = set()
for(i,j) in cpairs:
    if jacsim(sigmat[i], sigmat[j]) > threshold:
        realcpairs.add((i,j))

#realcpairs

Our LSH algorithm is now finished: it returns candidate pairs that we can work with. These potential pairs will now be used to find the official duplicate pairs. This will be considered in the code below. As products can only be classified as duplicate if they come from different stores, we remove the candidate pairs having the same Web shop. 

For identification of actual duplicates, we measure the Eucledian distance between the binary vectors of the potential pairs. We again have to specify a threshold, in such a way that some but not all pairs will be labeled as actual pairs. 

In [362]:
# Create function to calculate Eucledian distance for vectors in matrix

def EucDist(vec1, vec2):
    a = list(vec1)
    b = list(vec2)
    
    c = [ai - bi for ai, bi in zip(a, b)]
    dist = np.linalg.norm(c)
    
    return dist

testEUC = EucDist(Binary_Vector_Matrix.iloc[1], Binary_Vector_Matrix.iloc[6])
testEUC

5.0

In [ ]:
# Creating function for similarity measure

def preddups(binmat, obs, threshold_euc, realcpairs, include_dist = False):
    shop = dataframe[obs]['shop'] #extract shops from dataframe
    
    # Search within LSH-pairs
    pairs = []
    
    for val in realcpairs: #remove candidate pairs coming from same Web shop
        if val[0] == obs:
            if dataframe[val[1]]['shop'] != shop: 
                pairs.append(val[1]) 
            else:
                None
        else:
            None
        
    eucpairs = []    
    for pair in pairs: 
        dist = EucDist(binmat.iloc[obs], binmat.iloc[pair])
        if dist < threshold_euc:
            if include_dist == True:
                eucpairs.append((pair, dist))
            else:
                eucpairs.append(pair)    
    
    return eucpairs

testpreddups = actualduplicates(Binary_Vector_Matrix, 15, 5, realcpairs)
#testpreddups

Now that we have created the useful functions, we check our predictions. Next, we are curious to know the total number of predictions, the number of correct predictions and the number of duplicates found.

In [378]:
# Checking predictions
predictions = pd.DataFrame(columns=['product','predicted_duplicates','true_duplicates']) #create dataframe

for product in tqdm(range(len(dataframe))):
    true_duplicates = []
    predicted_duplicates = preddups(Binary_Vector_Matrix, product, 3.5, realcpairs)
    
    for item in range(len(dataframe)):
        if (dataframe[item]['modelID'] == dataframe[product]['modelID']) & (item!=product):
            true_duplicates.append(item)
    
    predictions = predictions.append({'product':product, 'predicted_duplicates':predicted_duplicates, 'true_duplicates':true_duplicates}, 
                 ignore_index = True)
    

100%|██████████| 1624/1624 [00:37<00:00, 42.96it/s]


In [373]:
predictions.head(10)

,product,predicted_duplicates,true_duplicates
0,0,[],[]
1,1,"[1216, 974, 12, 1107, 1154, 1439, 1314]",[]
2,2,[],[]
3,3,[],[]
4,4,[],[]
5,5,"[1328, 1085]",[]
6,6,[],[]
7,7,[],[]
8,8,[],[]
9,9,"[235, 716, 1443, 715, 565, 477, 713, 485, 328,...",[10]


In [377]:
# Calculating predictions and duplicates
totalpreds = 0
correctpreds = 0
totaldups = 0

for i in range(len(predictions)):
    for prediction in predictions.iloc[i]['predicted_duplicates']:     
        totalpreds+=1
        if prediction in predictions.iloc[i]['true_duplicates']:
            correctpreds +=1
    for duplicate in predictions.iloc[i]['true_duplicates']: 
        totaldups+=1
            
print(totaldups, totalpreds, correctpreds, (correctpreds/totalpreds))

798 5579 178 0.03190535938340204


The final step for us to take is to evaluate the algorithm. We do this by using a bootstrap function. From the original dataset, we know what the true duplicates are (based on the modelID). Hence, we use this for evaluation. 

In [388]:
# Generating true pairs from dataset
TruePair = []

for i in range(len(dataframe)):
    for j in range(len(dataframe)):
        if(dataframe[i]['modelID']==dataframe[j]['modelID']) & (i !=j):
            if (j,i) not in TruePair:
                TruePair.append((i,j))
            else:
                None
                
len(TruePair)

399

In [391]:
# Creating function for generating true pairs
def GenTruePair(dataframe):
    TruePair = []

    for i in range(len(dataframe)):
        for j in range(len(dataframe)):
            if(dataframe[i]['modelID']==dataframe[j]['modelID']) & (i !=j):
                if (j,i) not in TruePair:
                    TruePair.append((i,j))
                else:
                    None 
    return TruePair

In [407]:
# Defining bootstrap function for evaluation

randomlist = sample(range(0, 1623), int(1624*0.63))

def bootstrap(truepairs, dataframe, binmat, threshold_euc, realcpairs, reps): 
    dups = set()
    for p in truepairs:
        dups.add(p[0])
        dups.add(p[1])

    total_dups = len(dups)

    for i in tqdm(range(reps)): #do specified number of bootstrap repetitions

        total = range(0,1624) #product range
        train = sample(range(0, 1623), int(1624*0.63)) #63% of original data
        print(train[0])
        test = [x for x in total if x not in randomlist]

        train_datalist = []
        for num in train:
            train_datalist.append(dataframe[num])

        test_datalist = []
        for num in test:
            test_datalist.append(dataframe[num])

        train_pairs = GenTruePair(train_datalist)

        total_preds = 0 #initialize
        correct_preds = 0 #initialize

        for product in test: #evaluate predictions
            predicted_dups = preddups(Binary_Vector_Matrix, product, threshold_euc, realcpairs)
            total_preds = total_preds + len(predicted_dups)

            for pd in predicted_dups:
                if ((pd, product) in truepairs) | ((product, pd) in truepairs):
                    correct_preds = correct_preds + 1
        
        print(total_dups, total_preds, correct_preds)

In [410]:
threshold_euc = 5.0
reps = 5

bootstrap(TruePair, dataframe, Binary_Vector_Matrix, threshold_euc, realcpairs, 5)

  0%|          | 0/5 [00:00<?, ?it/s]

46
691 8550 95


 20%|██        | 1/5 [00:14<00:56, 14.17s/it]

638
691 8550 95


 40%|████      | 2/5 [00:25<00:40, 13.43s/it]

...

 60%|██████    | 3/5 [00:37<00:25, 12.92s/it]

23
691 8550 95


 80%|████████  | 4/5 [00:50<00:12, 12.99s/it]

21
691 8550 95


100%|██████████| 5/5 [01:03<00:00, 12.69s/it]
